## Import important Libaries

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from timeit import default_timer as timer
from tqdm.auto import tqdm
from helper_functions import *

In [ ]:
transform_0 = torchvision.transforms.Compose([torchvision.transforms.Grayscale(),
                                            torchvision.transforms.ToTensor()])

## Get data ready

In [ ]:
training_data = datasets.CIFAR10(root= "data", 
                                train=True, # get training data
                                download=True, # download data if it doesn't exist on disk
                                transform=transform_0, # images come as PIL format, we want to turn into Torch tensors
                                target_transform=None) # you can transform labels as well)

testing_data = datasets.CIFAR10(root= "data", 
                                train=False, # get training data
                                download=True, # download data if it doesn't exist on disk
                                transform=transform_0, # images come as PIL format, we want to turn into Torch tensors
                                target_transform=None) # you can transform labels as well)



In [ ]:
image, label = training_data[100]
image,label
image.shape, training_data.classes

In [ ]:
# Plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(training_data), size=[1]).item()
    img, label = training_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title(training_data.classes[label])
    plt.axis(False);

In [ ]:
BATCH_SIZE = 32

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=False)

print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

In [ ]:
training_images_batch, training_labels_batch = next(iter(train_dataloader))
print(training_labels_batch)
print(training_images_batch.shape)

In [ ]:
# Show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(training_images_batch), size=[1]).item()
img, label = training_images_batch[random_idx], training_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(training_data.classes[label])
plt.axis("Off");
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

In [ ]:
from models import mnist10_modelv0
torch.manual_seed(42)

model_1 = mnist10_modelv0(
    input_shape= training_images_batch.shape[2] * training_images_batch.shape[3],
    output_shape= len(training_data.classes)
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_1.parameters(), lr = 0.2)

# Setup device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"

model_1.to(device)
next(model_1.parameters()).device

In [ ]:
# Set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_gpu = timer()

# Set the number of epochs (we'll keep this small for faster training times)
epochs = 20

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch+1}\n---------")
    train_step(data_loader=train_dataloader, 
        model=model_1, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device = device
    )
    test_step(data_loader=test_dataloader,
        model=model_1,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device = device
    )

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)



In [ ]:
X,y = next(iter(test_dataloader))
print(y,X[0].shape)
y_pred = model_1(X.to(device)).argmax(dim=1)
print(y_pred)

In [ ]:
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    fig.add_subplot(rows,cols,i)
    plt.imshow(X[i].squeeze(), cmap="gray")
    if training_data.classes[y[i]] != training_data.classes[y_pred[i]]:

        plt.title(f"{training_data.classes[y[i]]}/{training_data.classes[y_pred[i]]}", color="red")
    else:
        plt.title(f"{training_data.classes[y[i]]}/{training_data.classes[y_pred[i]]}", color = "green")

    plt.axis(False);

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from models import mnistt10_CNN

In [ ]:
# Compute confusion matrix
y_pred = y_pred.to('cpu')
cm = confusion_matrix(y, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=training_data.classes, 
            yticklabels=training_data.classes)

plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
model_2 = mnistt10_CNN(input_shape=1,
                        hidden_units=10,
                        output_shape=len(training_data.classes)).to(device)

model_2

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_2.parameters(), lr = 0.1, weight_decay=0.001)

In [ ]:
# Train and test model 
epochs = 10
train_time_start_model_2 = timer()

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader, 
        model=model_2, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device=device
    )
    test_step(data_loader=test_dataloader,
        model=model_2,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device
    )

train_time_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_time_start_model_2,
                                           end=train_time_end_model_2,
                                           device=device)

In [ ]:
X,y = next(iter(test_dataloader))
print(y,X[0].shape)
y_pred = model_2(X.to(device)).argmax(dim=1)
print(y_pred)

In [ ]:
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    fig.add_subplot(rows,cols,i)
    plt.imshow(X[i].squeeze(), cmap="gray")
    if training_data.classes[y[i]] != training_data.classes[y_pred[i]]:

        plt.title(f"{training_data.classes[y[i]]}/{training_data.classes[y_pred[i]]}", color="red")
    else:
        plt.title(f"{training_data.classes[y[i]]}/{training_data.classes[y_pred[i]]}", color = "green")

    plt.axis(False);

In [ ]:
MODEL_PATH = Path("models_dic")

# Create model save path
MODEL_NAME = "03_pytorch_computer_vision_model_2.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_2.state_dict(), # only saving the state_dict() only saves the learned parameters
           f=MODEL_SAVE_PATH)